In [4]:
%load_ext autoreload
%autoreload 2

import numpy as np
import matplotlib.pyplot as plt
import re
import os

from result_utils import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Common function

In [5]:
handover_type = 'CU-1'
sim_duration = 60
results_dir = '../results/'
satellites_num = 200
AS = [[1, satellites_num + 36]]
gw_indices = [x for x in range(satellites_num + 1, satellites_num + 5)]
cell_indices = [x for x in range(satellites_num + 5, satellites_num + 37)]

topo_dir = './starlink-10-20-550-53-grid-LeastDelay'
change_file = topo_dir + '/Topo_leo_change.txt'

# assignments configuration
assignments_df = np.genfromtxt('./sim_configs/assignment.csv', delimiter=',', skip_header=1)
assignments_time = assignments_df[:, 0]
assignments = assignments_df[:, 1:].astype(int)

# user demands
demands_df = np.genfromtxt('./sim_configs/user_demand.csv', delimiter=',', skip_header=1)
demands_time = demands_df[:, 0]
demands = demands_df[:, 1:]

demands = demands[0]*10

change_matrix, change_time = construct_change_matrix(len(cell_indices), change_file, cell_indices, sim_duration)

Time: 3:
Add link: 167 217
Del link: 168 217
Time: 13:
Add link: 167 227
Del link: 168 227
Time: 17:
Add link: 53 210
Del link: 168 210
Time: 19:
Time: 21:
Add link: 167 228
Del link: 168 228
Time: 23:
Add link: 167 226
Del link: 168 226
Time: 28:
Add link: 167 223
Del link: 168 223
Time: 31:
Add link: 167 230
Del link: 168 230
Time: 32:
Del link: 53 210
Time: 34:
Add link: 167 205
Add link: 167 215
Del link: 168 215
Time: 35:
Del link: 168 225
Time: 37:
Add link: 168 219
Del link: 53 219
Time: 38:
Del link: 168 205
Time: 39:
Add link: 167 222
Del link: 168 222
Time: 41:
Add link: 167 221
Del link: 168 221
Time: 44:
Add link: 167 211
Del link: 168 211
Time: 46:
Add link: 167 208
Del link: 168 208
Time: 48:
Del link: 53 206
Time: 49:
Add link: 167 212
Del link: 168 212
Time: 51:
Del link: 168 213
Time: 53:
Add link: 167 225
Time: 54:
Add link: 167 232
Del link: 168 232
Time: 56:
Add link: 167 216
Del link: 168 216
Time: 59:
Add link: 167 209
Del link: 168 209
Time: 60:


## Throughput

In [6]:
# Load the data
total_transfer = 0
total_transfer_per_cell = []

total_demands_per_cell = []
total_demands = 0

for cell in cell_indices:
    perf_file = results_dir + 'gw_4_fix_demands/iperf/iperf_' + str(cell) + '_results.txt'
    sections = extract_perf_sections(perf_file)
    if sections is None:
        continue
    transferred_list, trans_rates_list = parse_perf_results(sections)
    
    transferred_list = handover_through_time(cell, gw_indices[assignments[0][cell_indices.index(cell)]],
                                             change_matrix, change_time, cell_indices, transferred_list,
                                             trans_rates_list, handover_type, topo_dir)
            
            
    transferred = aggregate_results(transferred_list)
    total_transfer_per_cell.append(transferred)

total_transfer = sum(total_transfer_per_cell)

total_demands_per_cell = demands * 56
total_demands = sum(total_demands_per_cell)

TypeError: expected str, bytes or os.PathLike object, not tuple

In [ ]:
print ("Total transfer: " + str(total_transfer) + " MBytes")
print ("Total demands: " + str(total_demands) + " MBytes")
print ("Total served rate: " + str(total_transfer/total_demands*100) + " %")

In [ ]:
serve_rate_per_cell = total_transfer_per_cell / total_demands_per_cell * 100
# Calculate the CDF

sorted_serve_rate = np.sort(serve_rate_per_cell)
cdf = np.arange(1, len(sorted_serve_rate) + 1) / len(sorted_serve_rate)

# Plot the CDF
plt.plot(sorted_serve_rate, cdf, label='Total Transfer')
plt.xlabel('Served rate (%)')
plt.ylabel('CDF')
plt.legend()
plt.title('CDF of Total Transfer and Total Demands')
plt.show()

## number of hops

In [ ]:
results_dir = '../results/gw_4_fix_demands/traceroute/'
file_list = os.listdir(results_dir)
traceroute_files = [file for file in file_list if file.startswith('traceroute_')]

total_hops = 0

avg_hops_per_cell = [0 for x in cell_indices]
success_links = [0 for x in cell_indices]

for file in traceroute_files:
    cell = int(file.split('_')[2])
    with open(results_dir + file, 'r') as f:
        traceroute_output = f.read()
    traceroute_lines = traceroute_output.split('\n')[:-1]
    hops = 0
    failed = 0
    if len(traceroute_lines) < 2:
        failed = 1
        continue

    for line in traceroute_lines[1:]:
        if '*' not in line or ('*' in line and '*' not in traceroute_lines[-1]):
            hops += 1
        elif '*' in line and '*' in traceroute_lines[-1]:
            failed = 1
            break
        
    if failed == 0:
        avg_hops_per_cell[cell_indices.index(cell)] += hops
        success_links[cell_indices.index(cell)] += 1
        
    if 2 <= len(traceroute_lines) < 3:
        print (file)

In [ ]:
import numpy as np

import matplotlib.pyplot as plt

# Calculate the CDF
sorted_avg_hops = np.sort(avg_hops_per_cell/np.array(success_links))
cdf = np.arange(1, len(sorted_avg_hops) + 1) / len(sorted_avg_hops)

# Plot the CDF
plt.plot(sorted_avg_hops, cdf, label='Avg Hops per Cell')
plt.xlabel('Avg Hops')
plt.ylabel('CDF')
plt.legend()
plt.title('CDF of Avg Hops per Cell over Success Links')
plt.show()

## links load

In [ ]:
results_dir = '../results/gw_4_fix_demands/traceroute/'
file_list = os.listdir(results_dir)
traceroute_files = [file for file in file_list if file.startswith('traceroute_')]

total_hops = 0

sat_loads = [0 for x in range(200)]
sat_links = [0 for x in range(200)]

for file in traceroute_files:
    

## Network delay

## Unconneted cell rate